In [ ]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
from tqdm import tqdm_notebook as tqdm
from glob import glob

In [ ]:
import torch as T
import torch.nn as nn
import torchvision as TV

In [ ]:
class Model_VE(nn.Module):
    def __init__(self):
        super(Model_VE, self).__init__()
        
        res = TV.models.resnet50(pretrained=True)
        self.res = nn.Sequential(*(list(res.children())[:-2]+[nn.AvgPool2d(4, 1)]))
        
    def forward(self, img):
        feat = self.res(img)
        feat = feat.reshape((feat.shape[0], -1))
        
        return feat
    
model = Model_VE().cuda()
feat = model(T.rand((1, 3, 160, 120)).cuda())
print(feat.shape)

In [ ]:
TRANS = TV.transforms.Compose([TV.transforms.ToTensor(), 
                               TV.transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                       std=[0.229, 0.224, 0.225])])

In [ ]:
from PIL import Image

feature = dict()
for i in tqdm(range(71, 111)):
    feature[i] = []
    
    l = len(glob('frame/%d/*'%(i)))
    
    for j in range(1, l+1):
        img = Image.open('frame/%d/%d.png' % (i, j))
        img = TRANS(img)
        img = img.view((1, )+img.shape)
        
        feat = model(img.cuda())
        feature[i].append(feat.cpu().detach().numpy()[0])

In [ ]:
import numpy as np

feature['START'] = np.ones((4096, ))
feature['END'] = np.zeros((4096, ))

In [ ]:
import pickle

pickle.dump(feature, open('feature.pkl', 'wb'))